<a href="https://colab.research.google.com/github/virf96/Basico/blob/main/GradienteEstoc%C3%A1sticoDescendente_CrossValidation_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

pd.set_option("max_columns", 500)
pd.set_option('display.float_format', lambda x: "{:,.4f}".format(x))

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
df = pd.read_csv("/content/drive/MyDrive/datasets/winequality-red.csv", sep=";")

In [80]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,"1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000"
mean,8.3196,0.5278,0.2710,2.5388,0.0875,15.8749,46.4678,0.9967,3.3111,0.6581,10.4230,5.6360
std,1.7411,0.1791,0.1948,1.4099,0.0471,10.4602,32.8953,0.0019,0.1544,0.1695,1.0657,0.8076
min,4.6000,0.1200,0.0000,0.9000,0.0120,1.0000,6.0000,0.9901,2.7400,0.3300,8.4000,3.0000
25%,7.1000,0.3900,0.0900,1.9000,0.0700,7.0000,22.0000,0.9956,3.2100,0.5500,9.5000,5.0000
50%,7.9000,0.5200,0.2600,2.2000,0.0790,14.0000,38.0000,0.9968,3.3100,0.6200,10.2000,6.0000
75%,9.2000,0.6400,0.4200,2.6000,0.0900,21.0000,62.0000,0.9978,3.4000,0.7300,11.1000,6.0000
max,15.9000,1.5800,1.0000,15.5000,0.6110,72.0000,289.0000,1.0037,4.0100,2.0000,14.9000,8.0000


In [5]:
X = df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 
        'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',]]
y = df[["quality"]]

### Escalamiento

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1199, 11), (400, 11), (1199, 1), (400, 1))

In [8]:
Sc = MinMaxScaler()
Sc.fit(X_train)
Xs = Sc.fit_transform(X_train)


Sc_y = MinMaxScaler()
Sc_y.fit(y_train)
ys = Sc_y.transform(y_train)

In [9]:
pd.DataFrame(ys)

,0
0,0.6000
1,0.4000
2,0.8000
3,0.4000
4,0.4000
...,...
1194,0.6000
1195,0.6000
1196,0.4000
1197,0.6000


## Modelado 

### Regresión

####Cross Validation Sin escalamiento

In [10]:
sgdr = SGDRegressor()
sgdr.fit(X_train, y_train)
ls_medias = cross_val_score(estimator=sgdr, X=X_train, y=y_train, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(-9.538516160387163e+24, 1.1339514025759306e+25)

#### Cross Validation Con escalamiento

In [11]:
sgdr = SGDRegressor()
sgdr.fit(Xs, ys)
ls_medias = cross_val_score(estimator=sgdr, X=Xs, y=ys, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.18304775098354156, 0.005284578972054146)

In [ ]:
SGDRegressor?

# GridSearchCV

In [16]:
param_grid = {"loss": ["squared_loss", "huber"],
              "penalty": ["l2", "l1", "elasticnet"],
              "alpha": [x/100 for x in range(10)],
              "epsilon": [x/10 for x in range(10)],
              "learning_rate": ["constant", "invscaling", "adaptive"],
             }

In [17]:
param_grid

{'alpha': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09],
 'epsilon': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'learning_rate': ['constant', 'invscaling', 'adaptive'],
 'loss': ['squared_loss', 'huber'],
 'penalty': ['l2', 'l1', 'elasticnet']}

In [18]:
n_hyper = np.prod(list(map(len, param_grid.values())))

In [19]:
n_hyper

1800

In [30]:
model = SGDRegressor(alpha=0.01)
hyper1 = GridSearchCV(estimator=model, n_jobs=-1, scoring="r2", param_grid=param_grid, error_score=-1000, verbose=5)
hyper1.fit(Xs, ys)
hyper1.best_score_

Fitting 5 folds for each of 1800 candidates, totalling 9000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1660 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 4540 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 8572 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 9000 out of 9000 | elapsed:   27.7s finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.32535432928991026

In [31]:
hyper1.best_estimator_

SGDRegressor(alpha=0.0, average=False, early_stopping=False, epsilon=0.9,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='adaptive', loss='huber', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [32]:
Sc_y.inverse_transform([hyper1.best_estimator_.intercept_])

array([[4.78273078]])

In [33]:
hyper1.best_estimator_.coef_

array([ 0.11725937, -0.19995896,  0.07751101, -0.00474232, -0.06803587,
        0.04699902, -0.08720507, -0.02157565,  0.05490332,  0.19096753,
        0.34312044])

#### Hyperparametrización

##### Sin escalamiento

In [24]:
%%time
model = SGDRegressor()
hyper = RandomizedSearchCV(estimator=model, n_jobs=-1, scoring="r2", param_distributions=param_grid, error_score=-1000, verbose=1, n_iter=int(n_hyper*.25), random_state=777)
hyper.fit(X_train, y_train)
hyper.best_score_

Fitting 5 folds for each of 450 candidates, totalling 2250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 1020 tasks      | elapsed:    7.9s


CPU times: user 1.86 s, sys: 24.4 ms, total: 1.88 s
Wall time: 16.5 s


[Parallel(n_jobs=-1)]: Done 2250 out of 2250 | elapsed:   16.5s finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
hyper.best_score_

0.3011658821974793

##### Con escalamiento

In [26]:
%%time
model = SGDRegressor()
hyper = RandomizedSearchCV(estimator=model, n_jobs=-1, scoring="r2", param_distributions=param_grid, error_score=-1000, verbose=1, n_iter=int(n_hyper*.25), random_state=777)
hyper.fit(Xs, ys)
hyper.best_score_

Fitting 5 folds for each of 450 candidates, totalling 2250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 956 tasks      | elapsed:    3.0s


CPU times: user 1.64 s, sys: 33.3 ms, total: 1.68 s
Wall time: 6.66 s


[Parallel(n_jobs=-1)]: Done 2250 out of 2250 | elapsed:    6.6s finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
hyper.best_score_

0.3252999004305105

In [28]:
X_train.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')

In [29]:
hyper.best_estimator_.coef_

array([ 0.11779266, -0.19828558,  0.07592568, -0.00528184, -0.06891933,
        0.0459189 , -0.08894154, -0.02155985,  0.05392006,  0.19076181,
        0.34342578])

# Modelo Ganador

In [35]:
SGD_GridSearch=hyper1.best_estimator_

In [36]:
#Escalamos el set de prueba
Xt=Sc.transform(X_test)

In [39]:
#Hacemos las predicciones regresandolas a su escala original para comparar con el y_test
predicciones=Sc_y.inverse_transform(SGD_GridSearch.predict(Xt).reshape(-1, 1))

In [41]:
#Veamos su performance
from sklearn.metrics import r2_score
r2_score(y_pred=predicciones, y_true=y_test)

0.29512805441003453

In [44]:
SGD_GridSearch.score(Xt, Sc_y.transform(y_test.values.reshape(-1,1)))

0.29512805441003487

# Clasificación 

In [106]:
#Creamos una variable binaria para plantear un modelo de clasificación
y_train_c = (y_train >= y_train.mean())*1
y_test_c = (y_test >= y_train.mean())*1

In [107]:
y_train_c

,y_c
1293,0
642,0
1545,1
1135,1
336,1
...,...
711,0
29,1
1334,0
148,1


In [109]:
#Escalamos
Sc = MinMaxScaler()
Sc.fit(X_train)
Xs=Sc.transform(X_train)

## Cross Validation

### Sin escalar

In [110]:
sgdc = SGDClassifier()
sgdc.fit(X_train, y_train_c)
ls_medias = cross_val_score(estimator=sgdc, X=X_train, y = y_train_c, cv = 4, n_jobs=-1, scoring="roc_auc")
np.mean(ls_medias), np.std(ls_medias)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.7207254001125041, 0.09592802014943365)

### Escalando

In [111]:
sgdc = SGDClassifier()
sgdc.fit(Xs, y_train_c)
ls_medias = cross_val_score(estimator=sgdc, X=X_train, y = y_train_c, cv = 4, n_jobs=-1, scoring="roc_auc")
np.mean(ls_medias), np.std(ls_medias)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(0.6631475055809541, 0.12402040644500635)

In [ ]:
SGDClassifier?

## Randomized Search CV

In [112]:
param_grid = {"loss": ["hinge", "log", "modified_huber", "squared_hinge", "perceptron"],
              "penalty": ["l2", "l1", "elasticnet"],
              "alpha": [x/100+0.01 for x in range(100)],
              "l1_ratio": [x/10 for x in range(10)],
              "learning_rate": ["constant", "optimal", "invscaling", "adaptive"],
              "eta0": [0.001]
             }

In [113]:
param_grid

{'alpha': [0.01,
  0.02,
  0.03,
  0.04,
  0.05,
  0.060000000000000005,
  0.06999999999999999,
  0.08,
  0.09,
  0.09999999999999999,
  0.11,
  0.12,
  0.13,
  0.14,
  0.15000000000000002,
  0.16,
  0.17,
  0.18000000000000002,
  0.19,
  0.2,
  0.21000000000000002,
  0.22,
  0.23,
  0.24000000000000002,
  0.25,
  0.26,
  0.27,
  0.28,
  0.29000000000000004,
  0.3,
  0.31,
  0.32,
  0.33,
  0.34,
  0.35000000000000003,
  0.36,
  0.37,
  0.38,
  0.39,
  0.4,
  0.41000000000000003,
  0.42,
  0.43,
  0.44,
  0.45,
  0.46,
  0.47000000000000003,
  0.48,
  0.49,
  0.5,
  0.51,
  0.52,
  0.53,
  0.54,
  0.55,
  0.56,
  0.5700000000000001,
  0.58,
  0.59,
  0.6,
  0.61,
  0.62,
  0.63,
  0.64,
  0.65,
  0.66,
  0.67,
  0.68,
  0.6900000000000001,
  0.7,
  0.71,
  0.72,
  0.73,
  0.74,
  0.75,
  0.76,
  0.77,
  0.78,
  0.79,
  0.8,
  0.81,
  0.8200000000000001,
  0.83,
  0.84,
  0.85,
  0.86,
  0.87,
  0.88,
  0.89,
  0.9,
  0.91,
  0.92,
  0.93,
  0.9400000000000001,
  0.95,
  0.96,
  0.97,
 

In [114]:
n_hyper = np.prod(list(map(len, param_grid.values())))

In [115]:
n_hyper

60000

#### Hyperparametrización

##### Sin escalamiento

In [116]:
%%time
model = SGDClassifier()
hyper = RandomizedSearchCV(estimator=model, n_jobs=-1, scoring="roc_auc", param_distributions=param_grid, error_score=-1000, verbose=1, n_iter=int(n_hyper*.1), random_state=777, cv = 4)
hyper.fit(X_train, y_train_c)
hyper.best_score_

Fitting 4 folds for each of 6000 candidates, totalling 24000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 540 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 2940 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done 6940 tasks      | elapsed:   51.9s
[Parallel(n_jobs=-1)]: Done 12540 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 19740 tasks      | elapsed:  2.4min


CPU times: user 23.3 s, sys: 418 ms, total: 23.7 s
Wall time: 2min 57s


[Parallel(n_jobs=-1)]: Done 24000 out of 24000 | elapsed:  3.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [117]:
hyper.best_score_

0.798488458676742

##### Con escalamiento

In [118]:
%%time
model = SGDClassifier()
hyper = RandomizedSearchCV(estimator=model, n_jobs=-1, scoring="roc_auc", param_distributions=param_grid, error_score=-1000, verbose=1, n_iter=int(n_hyper*.1), random_state=777, cv=4)
hyper.fit(Xs, y_train_c.values.ravel())
hyper.best_score_

Fitting 4 folds for each of 6000 candidates, totalling 24000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 956 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 5756 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done 13756 tasks      | elapsed:   51.5s


CPU times: user 18.3 s, sys: 173 ms, total: 18.5 s
Wall time: 1min 29s


[Parallel(n_jobs=-1)]: Done 24000 out of 24000 | elapsed:  1.5min finished


In [119]:
hyper.best_score_

0.8134515914565994

In [121]:
hyper.best_estimator_

SGDClassifier(alpha=0.25, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.001, fit_intercept=True,
              l1_ratio=0.8, learning_rate='optimal', loss='perceptron',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

### A pesar de que en el Cross Validation escalar los variables no nos dio los mejores resultados, cuando se hace la búsqueda aleatoria con Grid Search supera el performance del set sin escalar

# Modelo Ganador

In [122]:
#Escalamos
Xt=Sc.transform(X_test)

In [125]:
predicciones=hyper.best_estimator_.predict(Xt)

In [126]:
#Notamos que el modelo se sobreajusto
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test_c, pd.Series(predicciones))

0.5207686918226404